# 🦠 AHC Resistance Prediction App

A standalone Gradio application for predicting antimicrobial resistance using the optimized AHC resistance model from DAY_2.

This notebook provides a simple interface for clinicians to input patient parameters and receive resistance predictions with clinical interpretation.

**Author:** ESCMID AI Course 2025  
**Model:** resistance_model_optimised.pkl  
**Purpose:** Clinical decision support tool for antimicrobial resistance prediction

## 1. Import Required Libraries

Import all necessary packages for the resistance prediction application.

In [ ]:
import gradio as gr
import joblib
import numpy as np
import os

print("📦 Required libraries imported successfully")
print(f"   - Gradio version: {gr.__version__}")
print(f"   - NumPy version: {np.__version__}")

## 2. Configuration and Setup

Define the model path and application configuration parameters.

In [ ]:
# Model configuration
MODEL_PATH = "../../DAY_2/MODELS/resistance_model_optimised.pkl"
APP_TITLE = "🦠 AHC Resistance Prediction Tool"
APP_DESCRIPTION = """
Clinical decision support tool for predicting antimicrobial resistance patterns.
Input patient demographics and clinical parameters to receive resistance predictions.
"""

print(f"🔧 Application Configuration:")
print(f"   - Title: {APP_TITLE}")
print(f"   - Model Path: {MODEL_PATH}")
print(f"   - Model Exists: {os.path.exists(MODEL_PATH)}")

## 3. AHC Resistance Predictor Class

Main prediction engine for AHC resistance using the optimized model from DAY_2.

In [ ]:
class AHCResistancePredictor:
    """
    Main prediction engine for AHC resistance using the optimized model.
    """
    
    def __init__(self, model_path: str):
        """Initialize the predictor with the trained model."""
        self.model = None
        self.feature_names = None
        self.load_model(model_path)
    
    def load_model(self, model_path: str):
        """Load the trained resistance model."""
        try:
            if os.path.exists(model_path):
                self.model = joblib.load(model_path)
                print(f"✅ Model loaded successfully from {model_path}")
                
                # Define expected feature names based on DAY_2 notebook
                self.feature_names = [
                    'Age', 'Sex_M', 'Ward_ICU', 'Ward_Medical', 'Ward_Surgical',
                    'Specimen_Blood', 'Specimen_Respiratory', 'Specimen_Urine', 'Specimen_Wound'
                ]
                
            else:
                print(f"❌ Model file not found: {model_path}")
                print("Please ensure DAY_2 notebook has been run to generate the model.")
                self.model = None
                
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            self.model = None

print("🏗️ AHCResistancePredictor class defined")

## 4. Input Preparation Methods

Methods for preparing patient data for model input with proper feature encoding.

In [ ]:
def prepare_input(self, age: int, sex: str, ward: str, specimen: str) -> np.ndarray:
    """
    Prepare input features for the model.
    
    Args:
        age: Patient age
        sex: Patient sex (Male/Female)
        ward: Hospital ward (ICU/Medical/Surgical/Other)
        specimen: Specimen type (Blood/Respiratory/Urine/Wound/Other)
    
    Returns:
        Prepared feature array for prediction
    """
    # Initialize feature vector
    features = np.zeros(len(self.feature_names))
    
    # Age (continuous variable)
    features[0] = age
    
    # Sex (binary encoding)
    if sex == "Male":
        features[1] = 1  # Sex_M
    
    # Ward (one-hot encoding)
    if ward == "ICU":
        features[2] = 1  # Ward_ICU
    elif ward == "Medical":
        features[3] = 1  # Ward_Medical
    elif ward == "Surgical":
        features[4] = 1  # Ward_Surgical
    # Other wards remain all zeros
    
    # Specimen (one-hot encoding)
    if specimen == "Blood":
        features[5] = 1  # Specimen_Blood
    elif specimen == "Respiratory":
        features[6] = 1  # Specimen_Respiratory
    elif specimen == "Urine":
        features[7] = 1  # Specimen_Urine
    elif specimen == "Wound":
        features[8] = 1  # Specimen_Wound
    # Other specimens remain all zeros
    
    return features.reshape(1, -1)

# Add method to the class
AHCResistancePredictor.prepare_input = prepare_input

print("🔧 Input preparation method added to predictor class")

## 5. Clinical Interpretation Helper Methods

Methods for generating clinical interpretations and risk assessments.

In [ ]:
def _get_clinical_interpretation(self, age: int, ward: str, specimen: str) -> str:
    """Generate clinical interpretation based on patient factors."""
    
    interpretation = f"**Patient Profile Analysis:**\n"
    interpretation += f"• Age group: {self._get_age_group(age)}\n"
    interpretation += f"• Clinical setting: {ward}\n"
    interpretation += f"• Specimen type: {specimen}\n\n"
    
    interpretation += f"**Risk Factors:**\n"
    
    # Age-related risk
    if age >= 65:
        interpretation += "• Advanced age (≥65) - increased resistance risk\n"
    elif age <= 18:
        interpretation += "• Pediatric patient - different resistance patterns\n"
    
    # Ward-related risk
    if ward == "ICU":
        interpretation += "• ICU setting - high antimicrobial pressure environment\n"
    elif ward == "Surgical":
        interpretation += "• Surgical ward - perioperative antibiotic exposure\n"
    
    # Specimen-related risk
    if specimen == "Blood":
        interpretation += "• Bloodstream infection - serious clinical condition\n"
    elif specimen == "Respiratory":
        interpretation += "• Respiratory specimen - potential hospital-acquired infection\n"
    
    return interpretation

def _get_age_group(self, age: int) -> str:
    """Categorize age groups for clinical interpretation."""
    if age <= 18:
        return "Pediatric (≤18)"
    elif age <= 64:
        return "Adult (19-64)"
    else:
        return "Elderly (≥65)"

# Add methods to the class
AHCResistancePredictor._get_clinical_interpretation = _get_clinical_interpretation
AHCResistancePredictor._get_age_group = _get_age_group

print("📊 Clinical interpretation methods added to predictor class")

## 6. Main Prediction Method

Core prediction method that combines model inference with clinical interpretation.

In [ ]:
def predict_resistance(self, age: int, sex: str, ward: str, specimen: str) -> dict:
    """
    Predict resistance probability and provide clinical interpretation.
    
    Returns:
        Dictionary with prediction results and clinical recommendations
    """
    if self.model is None:
        return {
            "error": "Model not available. Please check model file path.",
            "prediction": "N/A",
            "probability": 0.0,
            "risk_level": "Unknown",
            "recommendation": "Cannot provide recommendation without model."
        }
    
    try:
        # Prepare input features
        features = self.prepare_input(age, sex, ward, specimen)
        
        # Get prediction and probability
        prediction = self.model.predict(features)[0]
        probability = self.model.predict_proba(features)[0]
        
        # Resistance probability (assuming class 1 is resistant)
        resistance_prob = probability[1] if len(probability) > 1 else probability[0]
        
        # Risk stratification
        if resistance_prob >= 0.7:
            risk_level = "🔴 HIGH RISK"
            recommendation = "Consider alternative antibiotics. Avoid empirical therapy with high-risk agents."
        elif resistance_prob >= 0.4:
            risk_level = "🟡 MODERATE RISK"
            recommendation = "Use with caution. Consider susceptibility testing before treatment."
        else:
            risk_level = "🟢 LOW RISK"
            recommendation = "Low resistance probability. Standard therapy may be appropriate."
        
        # Clinical interpretation
        interpretation = self._get_clinical_interpretation(age, ward, specimen)
        
        return {
            "prediction": "Resistant" if prediction == 1 else "Susceptible",
            "probability": f"{resistance_prob:.1%}",
            "risk_level": risk_level,
            "recommendation": recommendation,
            "interpretation": interpretation,
            "confidence": f"{max(probability):.1%}"
        }
        
    except Exception as e:
        return {
            "error": f"Prediction error: {str(e)}",
            "prediction": "Error",
            "probability": "0.0%",
            "risk_level": "Error",
            "recommendation": "Please check input parameters."
        }

# Add method to the class
AHCResistancePredictor.predict_resistance = predict_resistance

print("🎯 Main prediction method added to predictor class")

## 7. Initialize the Predictor

Create an instance of the AHC resistance predictor with the trained model.

In [ ]:
# Initialize the predictor
predictor = AHCResistancePredictor(MODEL_PATH)

print("🚀 AHC Resistance Predictor initialized")
print(f"   Model Status: {'✅ Loaded' if predictor.model else '❌ Not Available'}")
if predictor.feature_names:
    print(f"   Features: {len(predictor.feature_names)} parameters")
    print(f"   Feature Names: {predictor.feature_names}")

## 8. Gradio Interface Function

Function to handle Gradio interface interactions and format outputs.

In [ ]:
def make_prediction(age, sex, ward, specimen):
    """
    Gradio interface function for making predictions.
    """
    result = predictor.predict_resistance(age, sex, ward, specimen)
    
    if "error" in result:
        return result["error"], "", "", "", ""
    
    # Format output for Gradio
    prediction_text = f"**Prediction:** {result['prediction']}\n**Probability:** {result['probability']}"
    risk_text = f"{result['risk_level']}"
    recommendation_text = result['recommendation']
    interpretation_text = result['interpretation']
    confidence_text = f"Model Confidence: {result['confidence']}"
    
    return prediction_text, risk_text, recommendation_text, interpretation_text, confidence_text

print("🎛️ Gradio interface function defined")

## 9. Create Gradio Interface

Build the interactive web interface with input controls and output displays.

In [ ]:
def create_interface():
    """Create and configure the Gradio interface."""
    
    with gr.Blocks(title=APP_TITLE, theme=gr.themes.Soft()) as interface:
        
        # Header
        gr.Markdown(f"# {APP_TITLE}")
        gr.Markdown(APP_DESCRIPTION)
        
        # Model status
        model_status = "🟢 Model Loaded" if predictor.model is not None else "🔴 Model Not Available"
        gr.Markdown(f"**Model Status:** {model_status}")
        
        with gr.Row():
            # Input column
            with gr.Column(scale=1):
                gr.Markdown("### Patient Information")
                
                age_input = gr.Slider(
                    minimum=0,
                    maximum=100,
                    value=45,
                    step=1,
                    label="Age (years)"
                )
                
                sex_input = gr.Radio(
                    choices=["Male", "Female"],
                    value="Female",
                    label="Sex"
                )
                
                ward_input = gr.Dropdown(
                    choices=["ICU", "Medical", "Surgical", "Other"],
                    value="Medical",
                    label="Hospital Ward"
                )
                
                specimen_input = gr.Dropdown(
                    choices=["Blood", "Respiratory", "Urine", "Wound", "Other"],
                    value="Urine",
                    label="Specimen Type"
                )
                
                predict_button = gr.Button("🔬 Predict Resistance", variant="primary")
            
            # Output column
            with gr.Column(scale=1):
                gr.Markdown("### Prediction Results")
                
                prediction_output = gr.Markdown(label="Prediction")
                risk_output = gr.Markdown(label="Risk Level")
                recommendation_output = gr.Markdown(label="Clinical Recommendation")
                
        # Detailed interpretation
        with gr.Row():
            with gr.Column():
                gr.Markdown("### Clinical Interpretation")
                interpretation_output = gr.Markdown(label="Detailed Analysis")
                confidence_output = gr.Markdown(label="Model Performance")
        
        # Quick test examples
        with gr.Row():
            gr.Markdown("### 🚀 Quick Test Examples")
            
            with gr.Column():
                gr.Markdown("**High-Risk Scenario**")
                high_risk_btn = gr.Button("ICU Patient, Blood Culture")
                
            with gr.Column():
                gr.Markdown("**Moderate-Risk Scenario**")
                mod_risk_btn = gr.Button("Elderly Patient, UTI")
                
            with gr.Column():
                gr.Markdown("**Low-Risk Scenario**")
                low_risk_btn = gr.Button("Young Adult, Outpatient")
        
        # Event handlers
        predict_button.click(
            fn=make_prediction,
            inputs=[age_input, sex_input, ward_input, specimen_input],
            outputs=[prediction_output, risk_output, recommendation_output, 
                    interpretation_output, confidence_output]
        )
        
        # Quick example handlers
        high_risk_btn.click(
            lambda: (75, "Male", "ICU", "Blood"),
            outputs=[age_input, sex_input, ward_input, specimen_input]
        )
        
        mod_risk_btn.click(
            lambda: (70, "Female", "Medical", "Urine"),
            outputs=[age_input, sex_input, ward_input, specimen_input]
        )
        
        low_risk_btn.click(
            lambda: (25, "Female", "Other", "Urine"),
            outputs=[age_input, sex_input, ward_input, specimen_input]
        )
        
        # Footer
        gr.Markdown("""
        ---
        **Disclaimer:** This tool is for educational and research purposes only. 
        Clinical decisions should always be made by qualified healthcare professionals 
        considering local antibiograms and patient-specific factors.
        """)
    
    return interface

print("🎨 Gradio interface creation function defined")

## 10. Launch the Application

Create and launch the AHC resistance prediction web application.

In [ ]:
# Create the interface
app = create_interface()

print("🚀 Starting AHC Resistance Prediction App...")
print(f"📁 Model path: {MODEL_PATH}")
print(f"🔧 Model status: {'Loaded' if predictor.model else 'Not available'}")
print("")
print("💡 Ready to launch! Run the next cell to start the web interface.")

## 11. Launch Options

Choose your preferred launch configuration for the web application.

In [ ]:
# Launch the app with sharing enabled
app.launch(
    share=True,
    show_error=True
)

print("🌐 Application launched successfully!")
print("   Click the provided URL to access the web interface")
print("   Use the quick test examples to verify functionality")

## 12. Testing and Validation

Test the predictor functionality programmatically to ensure correct operation.

In [ ]:
# Test the predictor with sample data
print("🧪 Testing AHC Resistance Predictor...")
print("")

# Test case 1: High-risk scenario
print("Test Case 1: High-Risk ICU Patient")
result1 = predictor.predict_resistance(75, "Male", "ICU", "Blood")
if "error" not in result1:
    print(f"   Prediction: {result1['prediction']}")
    print(f"   Probability: {result1['probability']}")
    print(f"   Risk Level: {result1['risk_level']}")
else:
    print(f"   Error: {result1['error']}")
print("")

# Test case 2: Low-risk scenario
print("Test Case 2: Low-Risk Young Patient")
result2 = predictor.predict_resistance(25, "Female", "Other", "Urine")
if "error" not in result2:
    print(f"   Prediction: {result2['prediction']}")
    print(f"   Probability: {result2['probability']}")
    print(f"   Risk Level: {result2['risk_level']}")
else:
    print(f"   Error: {result2['error']}")
print("")

print("✅ Testing completed")

---

## 📚 Usage Instructions

1. **Run all cells sequentially** to set up the application
2. **Launch the web interface** using cell 11
3. **Input patient parameters** in the web interface:
   - Age (0-100 years)
   - Sex (Male/Female)
   - Hospital Ward (ICU/Medical/Surgical/Other)
   - Specimen Type (Blood/Respiratory/Urine/Wound/Other)
4. **Click "Predict Resistance"** to get results
5. **Review the clinical interpretation** and recommendations

### 🎯 Quick Test Examples

Use the provided quick test buttons to populate the interface with:
- **High-Risk**: 75-year-old male, ICU, blood culture
- **Moderate-Risk**: 70-year-old female, medical ward, UTI
- **Low-Risk**: 25-year-old female, outpatient, UTI

### ⚠️ Important Notes

- Ensure the `resistance_model_optimised.pkl` file exists in the DAY_2 directory
- This tool is for educational and research purposes only
- Clinical decisions should always involve qualified healthcare professionals
- Consider local antibiograms and resistance patterns in clinical practice